In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
import torch

# Загрузка данных
data = pd.read_csv('processed_data.csv')  # замените на путь к вашему файлу

# Определение категорий
categories = [
    'Вопрос решен', 
    'Нравится качество выполнения заявки', 
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок', 
    'Понравилось выполнение заявки', 
    'Другое'
]

# Извлечение текстов и меток
texts = data['comment'].tolist()
labels = data[categories].values

# Разделение данных на обучающую и валидационную выборки
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Инициализация токенизатора
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Токенизация данных
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# Создание датасета для PyTorch
class CommentsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

train_dataset = CommentsDataset(train_encodings, train_labels)
val_dataset = CommentsDataset(val_encodings, val_labels)

# Загрузка модели для мультиярисовой классификации
model = XLMRobertaForSequenceClassification.from_pretrained(
    'xlm-roberta-base',
    num_labels=len(categories),
    problem_type="multi_label_classification"
)

# Настройка обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
)

# Функция для вычисления метрик (F1 по каждой категории и средний ROC-AUC)
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions))
    preds_binarized = preds > 0.5
    
    y_true = p.label_ids
    
    # F1-score для каждой категории
    f1_scores_per_category = [
        f1_score(y_true[:, i], preds_binarized[:, i].cpu().numpy()) for i in range(y_true.shape[1])
    ]
    
    # Средний F1-score (micro)
    f1_micro_value = f1_score(y_true, preds_binarized.cpu().numpy(), average='micro')
    
    # ROC-AUC по всем меткам (macro)
    try:
        roc_auc_value = roc_auc_score(y_true, preds.cpu().numpy(), average='macro')
    except ValueError:
        roc_auc_value = np.nan
    
    # Возвращаем словарь метрик для логирования
    return {
        **{f'f1_{categories[i]}': f1_scores_per_category[i] for i in range(len(categories))},
        'f1_micro': f1_micro_value,
        'roc_auc_mean': roc_auc_value,
    }

# Инициализация Trainer с метриками
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

# Предсказания на валидационной выборке
predictions_output = trainer.predict(val_dataset)
preds_probs = torch.sigmoid(torch.tensor(predictions_output.predictions)).cpu().numpy()
preds_binarized = preds_probs > 0.5

# Вывод отчетов по каждой категории
for i, category in enumerate(categories):
    print(f'Категория: {category}')
    print(classification_report(
        val_labels[:, i],
        preds_binarized[:, i].astype(int),
        target_names=['Нет', 'Да']
    ))

# Общий ROC-AUC по всем меткам (средний)
try:
    roc_auc_total = roc_auc_score(val_labels.flatten(), preds_probs.flatten())
except ValueError:
     roc_auc_total = np.nan

print(f"\nОбщий ROC-AUC (по всем меткам): {roc_auc_total:.4f}")
print(f"F1-score (micro): {f1_score(val_labels.flatten(), preds_binarized.flatten(), average='micro'):.4f}")

# Сохранение модели и токенизатора
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.602400
20,0.487400
30,0.510800
40,0.488500
50,0.499600
60,0.518100
70,0.475800
80,0.447400
90,0.438200
100,0.388600


d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Категория: Вопрос решен
              precision    recall  f1-score   support

         Нет       0.86      0.89      0.88       226
          Да       0.50      0.42      0.46        57

    accuracy                           0.80       283
   macro avg       0.68      0.66      0.67       283
weighted avg       0.79      0.80      0.79       283

Категория: Нравится качество выполнения заявки
              precision    recall  f1-score   support

         Нет       0.92      1.00      0.96       260
          Да       1.00      0.04      0.08        23

    accuracy                           0.92       283
   macro avg       0.96      0.52      0.52       283
weighted avg       0.93      0.92      0.89       283

Категория: Нравится качество работы сотрудников
              precision    recall  f1-score   support

         Нет       0.96      0.96      0.96       208
          Да       0.88      0.88      0.88        75

    accuracy                           0.94       283
   macro 

d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\sentencepiece.bpe.model',
 './saved_model\\added_tokens.json')